# File Creation for Zooplankton

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import os

from tqdm import tqdm
import matplotlib.pyplot as plt


## Datasets Preparation

In [ ]:
def datasets_preparation(name, units, coords):

    dataset = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'],
        attrs=dict(description='Integrated quantity of interest',
        long_name =name,
        units=units))
        
    return (dataset)

## File Creation

In [ ]:
def file_creation(variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path='/data/ibougoudis/MOAD/files/apr_jun.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})
    temp.close()
    

## Data Collection

In [ ]:
dict_month = {'jan': '01',
    'feb': '02',
    'mar': '03',
    'apr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'aug': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dec': '12'}

path = os.listdir('/results2/SalishSea/nowcast-green.202111/')

folders = [x for x in path if ((x[2:5]=='apr' and x[0:2] > '15') or x[2:5] == 'may' or x[2:5] == 'jun')]
if '17apr24.aside' in folders:
    folders.pop(folders.index('17apr24.aside')) # Removing an unnecessary directory (17apr24.aside)

indx_dates=(np.argsort(pd.to_datetime(folders, format="%d%b%y")))
folders = [folders[i] for i in indx_dates]

# Open the mesh mask
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
mask = mesh.tmask.to_numpy()

coords = dict(time_counter=pd.to_datetime(folders, format="%d%b%y"), y=mesh.y, x=mesh.x) 

micro_i_all = datasets_preparation('Z1 Concentration', 'mmol N m-2', coords)
meso_i_all = datasets_preparation('Z2 Concentration', 'mmol N m-2', coords)

micro_diat_graz_i_all = datasets_preparation('Z1 Grazing on Diatom', 's-1', coords)
micro_flag_graz_i_all = datasets_preparation('Z1 Grazing on Flagellate', 's-1', coords)
micro_pon_graz_i_all = datasets_preparation('Z1 Grazing on Pon', 's-1', coords)
micro_micro_graz_i_all = datasets_preparation('Z1 Grazing on Z1', 's-1', coords)

meso_diat_graz_i_all = datasets_preparation('Z2 Grazing on Diatom', 's-1', coords)
meso_flag_graz_i_all = datasets_preparation('Z2 Grazing on Flagellate', 's-1', coords)
meso_pon_graz_i_all = datasets_preparation('Z2 Grazing on Pon', 's-1', coords)
meso_micro_graz_i_all = datasets_preparation('Z2 Grazing on Z1', 's-1', coords)

print(len(folders))


## Calculations

In [ ]:
for i in tqdm(enumerate(folders)):

    ds_name = ('/results2/SalishSea/nowcast-green.202111/' + i[1] + '/SalishSea_1d_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]])+str(i[1][0:2]) + '_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]]) + str(i[1][0:2]) + '_grid_T.nc')
    ds_bio_name = ('/results2/SalishSea/nowcast-green.202111/' + i[1] + '/SalishSea_1d_'  + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]])+str(i[1][0:2]) + '_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]]) + str(i[1][0:2]) + '_biol_T.nc')
    ds_graz_name = ('/results2/SalishSea/nowcast-green.202111/' + i[1] + '/SalishSea_1d_'  + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]])+str(i[1][0:2]) + '_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]]) + str(i[1][0:2]) + '_graz_T.nc') 
    
    ds = xr.open_dataset (ds_name)
    ds_bio = xr.open_dataset (ds_bio_name)
    ds_graz = xr.open_dataset (ds_graz_name)

    micro_i_all[i[0]] =((ds_bio.microzooplankton.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])
    meso_i_all[i[0]] = ((ds_bio.mesozooplankton.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    micro_diat_graz_i_all[i[0]] = ((ds_graz.GRMICZDIAT.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / micro_i_all[i[0]]    
    micro_flag_graz_i_all[i[0]] = ((ds_graz.GRMICZPHY.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / micro_i_all[i[0]]
    micro_pon_graz_i_all[i[0]] = ((ds_graz.GRMICZPON.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / micro_i_all[i[0]]
    micro_micro_graz_i_all[i[0]] = ((ds_graz.GRMICZMICZ.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / micro_i_all[i[0]]
    
    meso_diat_graz_i_all[i[0]] = ((ds_graz.GRMESZDIAT.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / meso_i_all[i[0]]    
    meso_flag_graz_i_all[i[0]] = ((ds_graz.GRMESZPHY.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / meso_i_all[i[0]]
    meso_pon_graz_i_all[i[0]] = ((ds_graz.GRMESZPON.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / meso_i_all[i[0]]
    meso_micro_graz_i_all[i[0]] = ((ds_graz.GRMESZMICZ.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / meso_i_all[i[0]]
    

## Saving

In [ ]:
file_creation(micro_i_all, 'Z1')
file_creation(meso_i_all, 'Z2')

file_creation(micro_diat_graz_i_all, 'Z1_Diatom')
file_creation(micro_flag_graz_i_all, 'Z1_Flagellate')
file_creation(micro_pon_graz_i_all, 'Z1_Pon')
file_creation(micro_micro_graz_i_all, 'Z1_Z1')

file_creation(meso_diat_graz_i_all, 'Z2_Diatom')
file_creation(meso_flag_graz_i_all, 'Z2_Flagellate')
file_creation(meso_pon_graz_i_all, 'Z2_Pon')
file_creation(meso_micro_graz_i_all, 'Z2_Z1')
